Draft from before I knew about the existence of Dijkstra's

In [132]:
import numpy as np
np.set_printoptions(linewidth=np.inf)
lossmap = np.genfromtxt("input17.dat", int, delimiter=1)

In [1]:
with open("input17.dat", "r") as file:
    lossmap = file.readlines()
lossmap = [list(map(int,line.removesuffix("\n"))) for line in lossmap]
max_v, max_h = len(lossmap), len(lossmap[0])
# lossmap = {(i,j) : lossmap[i][j] for i in range(max_v) for j in range(max_h)} #makes it ~11% slower 

In [ ]:
with open("exampleinput17.dat", "r") as file:
    lossmap = file.readlines()
lossmap = [line.removesuffix("\n") for line in lossmap]
# incline = [[-i**2-j**2+(len(lossmap)-1)**2+(len(lossmap[0])-1)**2 for j in range(len(lossmap[0]))] for i in range(len(lossmap))]
# eff_lossmap = [[int(lossmap[i][j])+incline[i][j] for j in range(len(lossmap[0]))] for i in range(len(lossmap))]

In [5]:
lossmap = np.genfromtxt("exampleinput17.dat", int, delimiter=1)
incline = np.zeros(lossmap.shape, int)
for i in range(incline.shape[0]):
    for j in range(incline.shape[1]):
        # incline[i,j] = -i**2-j**2+(incline.shape[0]-1)**2+(incline.shape[1]-1)**2
        incline[i,j] = -i-j+(incline.shape[0]-1)**2+(incline.shape[1]-1)**2

In [2]:
import random
def progression(lossmap, min_loss):
    delta = {0: (0,1), 1: (1,0), 2: (0,-1), 3: (-1,0)} 
    loss = 0
    point, direc = (0,0), random.sample([0,1], 1)[0]
    path = [point]
    point = tuple(map(sum,zip(point,delta[direc])))
    path.append(point)
    loss += int(lossmap[point[0]][point[1]])

    while point != (len(lossmap)-1, len(lossmap[0])-1):
        possib_dirs = [0,1,2,3]
        possib_dirs.remove((direc+2)%4)
        if point[0]==0:
            possib_dirs.remove(3)
        if point[0]==len(lossmap)-1:
            possib_dirs.remove(1)
        if point[1]==0:
            possib_dirs.remove(2)
        if point[1]==len(lossmap[0])-1:
            possib_dirs.remove(0)
        if len({p[0] for p in path[-4:]})==1:
            possib_dirs = [_ for _ in possib_dirs if _ not in [0,2]]
        if len({p[1] for p in path[-4:]})==1:
            possib_dirs = [_ for _ in possib_dirs if _ not in [1,3]]
            
        direc = random.sample(possib_dirs, 1)[0]
        point = tuple(map(sum,zip(point,delta[direc])))
        loss += int(lossmap[point[0]][point[1]])
        path.append(point)
        if loss > min_loss:
            return None, None
    return loss, path

def staircase_prog(lossmap):
    delta = {0: (0,1), 1: (1,0), 2: (0,-1), 3: (-1,0)}
    loss = 0
    point, direc = (0,0), 1
    path = [point]

    while point != (len(lossmap)-1, len(lossmap[0])-1):          
        direc = (direc+1)%2 #ladder path for reference
        point = tuple(map(sum,zip(point,delta[direc])))
        loss += int(lossmap[point[0]][point[1]])
        path.append(point)

    return loss, path

def local_min_prog(lossmap):
    delta = {0: (0,1), 1: (1,0), 2: (0,-1), 3: (-1,0)}
    loss = 0
    point, direc = (0,0), 0
    if lossmap[1][0] < lossmap[0][1]:
        direc = 1
    path = [point]
    point = tuple(map(sum,zip(point,delta[direc])))
    path.append(point)
    loss += int(lossmap[point[0]][point[1]])

    while point != (len(lossmap)-1, len(lossmap[0])-1):
        possib_dirs = [0,1,2,3]
        possib_dirs.remove((direc+2)%4)
        if point[0]==0:
            possib_dirs.remove(3)
        if point[0]==len(lossmap)-1:
            possib_dirs.remove(1)
        if point[1]==0:
            possib_dirs.remove(2)
        if point[1]==len(lossmap[0])-1:
            possib_dirs.remove(0)
        if len({p[0] for p in path[-4:]})==1:
            possib_dirs = [_ for _ in possib_dirs if _ not in [0,2]]
        if len({p[1] for p in path[-4:]})==1:
            possib_dirs = [_ for _ in possib_dirs if _ not in [1,3]]
        points = {direc: int(lossmap[point[0]+delta[direc][0]][point[1]+delta[direc][1]]) for direc in possib_dirs}
        direc = list(points.keys())[list(points.values()).index(min(points.values()))]
        point = tuple(map(sum,zip(point,delta[direc])))
        while point in path[:-1]:
            del points[direc]
            if points == {}:
                return -loss, path
            direc = list(points.keys())[list(points.values()).index(min(points.values()))]
            point = tuple(map(sum,zip(point,delta[direc])))

        loss += int(lossmap[point[0]][point[1]])
        path.append(point)

    return loss, path


In [3]:
min_loss, min_path = staircase_prog(lossmap)
loss, path = local_min_prog(lossmap)
if loss >0:
    min_loss = loss
    min_path = path
for _ in range(1000):
    loss, path = progression(lossmap, min_loss)
    if loss is not None and loss < min_loss:
        min_loss = loss
        min_path = path
min_loss

133

In [6]:
def progr_w_prob(lossmap, penalties, min_loss):
    delta = {0: (0,1), 1: (1,0), 2: (0,-1), 3: (-1,0)}
    eff_lossmap = lossmap + penalties
    eff_loss = 0
    point, direc = (0,0), random.choice([0,1])
    path = [point]
    point = tuple(map(sum,zip(point,delta[direc])))
    path.append(point)
    eff_loss += int(eff_lossmap[point[0]][point[1]])

    while point != (len(eff_lossmap)-1, len(eff_lossmap[0])-1):
        next_dirs = [0,1,2,3]
        next_dirs.remove((direc+2)%4)
        if point[0]==0:
            next_dirs.remove(3)
        if point[0]==len(eff_lossmap)-1:
            next_dirs.remove(1)
        if point[1]==0:
            next_dirs.remove(2)
        if point[1]==len(eff_lossmap[0])-1:
            next_dirs.remove(0)
        if len({p[0] for p in path[-4:]})==1:
            next_dirs = [_ for _ in next_dirs if _ not in [0,2]]
        if len({p[1] for p in path[-4:]})==1:
            next_dirs = [_ for _ in next_dirs if _ not in [1,3]]

        next_pts = [tuple(map(sum,zip(point,delta[direc]))) for direc in next_dirs]
        next_losses = [int(eff_lossmap[point[0]][point[1]]) for point in next_pts]
        next_probs = [2**(-loss) for loss in next_losses]
        next_probs = [prob/sum(next_probs) for prob in next_probs]
        direc = random.choices(next_dirs, next_probs)[0]

        point = tuple(map(sum,zip(point,delta[direc])))
        eff_loss += int(eff_lossmap[point[0]][point[1]])
        path.append(point)
        
        loss = sum([lossmap[i][j] for (i,j) in path])
        if loss > min_loss:
            return loss, path, point
    return loss, path, eff_loss

In [7]:
import matplotlib.pyplot as plt
# min_loss = 10**10

for _ in range(1000):
    loss, path, eff_loss = progr_w_prob(lossmap, 2*incline, min_loss)
    if not isinstance(eff_loss, tuple):
        if loss < min_loss:
            min_loss = loss
            min_path = path
            
# plt.matshow([[((i,j) in path) for j in range(len(lossmap[0]))] for i in range(len(lossmap))]);
# plt.matshow([[((i,j) in min_path) for j in range(len(lossmap[0]))] for i in range(len(lossmap))]);
min_loss, eff_loss

(108, (10, 11))